## Imports

In [11]:
import os
import nibabel as nib
import glob

### Reading all BraTS21 examples, change the root_path to you base address

In [12]:
root_path = r'C:\Users\Mohamed Bushnaq\PycharmProjects\Slices\Brats21'             #Change this to dataset base address
data_list = sorted(glob.glob(root_path + '/*'))                                    #list of paths of the inside files

#### The following function is responsible for returning the indices of the brain of the volume that contains foreground voxels.

In [13]:
def find_brain_width_wise(dep, hei, i):        #cropping width wise
    slice2D = img.get_fdata()[:, i, :]
    for j in range(hei):
        for k in range(dep):
            if slice2D[j, k] != 0:
                return i
    return 0

def find_brain_height_wise(dep, wid, i):      #cropping height wise
    slice2D = img.get_fdata()[i, :, :]
    for j in range(wid):
        for k in range(dep):
            if slice2D[j, k] != 0:
                return i
    return 0

def find_brain_depth_wise(wid, hei, i):        #cropping depth wise
    slice2D = img.get_fdata()[:, :, i]
    for j in range(wid):
        for k in range(hei):
            if slice2D[j, k] != 0:
                return i
    return 0

#### Creating results folder that will contain the cropped volumes

In [14]:
# Create results folder
results_path = r'C:\Users\Mohamed Bushnaq\Graduation Projects GANs Conversion\Brats Cropped For GaNs Modality Conversion 150'
if not os.path.exists(results_path):
    os.makedirs(results_path)

#### Looping in every example and in every module in the volume to crop it to contain only the brain part

In [15]:
#Iterating for every example in the folder
for example in range(150):                   #150 -> The number of example I want to crop and save from the data
    modules_list = sorted(glob.glob(data_list[example] + '/*'))          #modules list of a single volume
    seg_vol_path = modules_list[0]                        #It doesn't matter which mode, just no seg (tumor), as we want the brain
    img = nib.load(seg_vol_path)                          #Flair volume (it doesn't matter)

    height, width, depth = img.shape
    filled_slices_width = []                              #slices indices with foreground values
    filled_slices_height = []
    filled_slices_depth = []

    #Iterating in the dimension of interest, which we will extract foreground slices from, repeating it in the 3 dimension (3D Volume)
    for i in range(depth):
        depth_idx = find_brain_depth_wise(width, height, i)
        if depth_idx != 0:
            filled_slices_depth.append(depth_idx)

    for i in range(width):
        width_idx = find_brain_width_wise(depth, height, i)
        if width_idx != 0:
            filled_slices_width.append(width_idx)

    for i in range(height):
        height_idx = find_brain_height_wise(depth, width, i)
        if height_idx != 0:
            filled_slices_height.append(height_idx)


    min_wid_idx, max_wid_idx = filled_slices_width[0], filled_slices_width[-1]
    min_hei_idx, max_hei_idx = filled_slices_height[0], filled_slices_height[-1]
    min_dep_idx, max_dep_idx = filled_slices_depth[0], filled_slices_depth[-1]

    #Cropping step, iterating for every module in the same example to be cropped.
    for module_path in modules_list:
        module_vol = nib.load(module_path)
        cropped_vol = module_vol.get_fdata()[min_hei_idx : (max_hei_idx+1),
                                             min_wid_idx : (max_wid_idx+1),
                                             min_dep_idx : (max_dep_idx+1)]

        nifti_img =  nib.Nifti1Image(cropped_vol, module_vol.affine)            # to save this 3D (ndarry) numpy

        #to make the naming of the resulted folders and files the same as the original naming
        vol_new_path = results_path + '/' + data_list[example].split('\\')[-1]
        if not os.path.exists(vol_new_path):
            os.makedirs(vol_new_path)

        module_new_path = vol_new_path + '/' + module_path.split('\\')[-1]
        nib.save(nifti_img, module_new_path)

### Testing random module

In [17]:
test_path = r'C:\Users\Mohamed Bushnaq\Graduation Projects GANs Conversion\Brats Cropped For GaNs Modality Conversion 150\BraTS2021_00019\BraTS2021_00019_t1.nii.gz'
img_test = nib.load(test_path)

img_test.shape

(142, 177, 142)

#### Image shape was (240, 240, 155) Before cropping, now it's (142, 177, 142) for this example (Random)